In [ ]:
!pip install OpenAI tenacity

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00


In [3]:
import json
import os
import matplotlib.pyplot as plt

def read_jsonl(path):
    examples = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                try:
                    ex = json.loads(line)
                    examples.append(ex)
                except Exception as e:
                    print(e)
    return examples

def get_image_from_problem(problem):
    img_folder='TRAINING'
    img_list=[]
    frame_list=[]
    reason_list=[]
    ids=[]
    for i in read_jsonl('predictions_whole/articulation-annotations.jsonl'):
        for j in i['articulations']:
            if problem in j['reasoning']:
                img_list.append(i['id'])
                frame_list.append(j['text'])
                reason_list.append(j['reasoning'])
                ids.append(i['id'])
    print(len(img_list))
    for x, f_name in enumerate(img_list):
      try:
        image_path = os.path.join(img_folder, f_name)
      except:
        print('Not found')
      image = plt.imread(image_path)
      print(frame_list[x])
      print(reason_list[x])
      plt.imshow(image)
      plt.title(frame_list[x])
      plt.xlabel(ids[x])
      #plt.axis('off')
      plt.show()

def get_frame_from_problem(problem):
    img_folder='TRAINING'
    img_list=[]
    frame_list=[]
    reason_list=[]
    ids=[]
    for i in read_jsonl('predictions_whole/articulation-annotations.jsonl'):
        for j in i['articulations']:
            if problem in j['reasoning']:
                img_list.append(i['id'])
                frame_list.append(j['text'])
                reason_list.append(j['reasoning'])
                ids.append(i['id'])
    return frame_list, reason_list

prblm_dict={}
reason_dict={}
with open('problems.txt', 'r') as f:
    problems = f.read().splitlines()
    problems=[problem.split(':')[0] for problem in problems]
for problem in problems:
    problem2='problem of '+ problem + ' arises'
    frame_list,reason_list=get_frame_from_problem(problem2)
    prblm_dict[problem]=frame_list
    reason_dict[problem]=reason_list

In [ ]:
#another method of calling openai
from openai import OpenAI, BadRequestError
from openai.types.chat import ChatCompletion
from tenacity import retry, stop_after_attempt, wait_random_exponential
import time
from typing import Optional
import json
import base64

class MinimumDelay:
    def __init__(self, delay: float | int):
        self.delay = delay
        self.start = None

    def __enter__(self):
        self.start = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        end = time.time()
        seconds = end - self.start
        if self.delay > seconds:
            time.sleep(self.delay - seconds)

@retry(wait=wait_random_exponential(min=1, max=90), stop=stop_after_attempt(3))
def chat(client: OpenAI, delay: float | int, **kwargs) -> ChatCompletion | None:
    try:
        with MinimumDelay(delay):
            return client.chat.completions.create(**kwargs)
    except BadRequestError as e:
        print(f"Bad Request: {e}")
        if "safety" in e.message:
            return None
        raise e
    except Exception as e:
        print(f"Exception: {e}")
        raise e

def read_jsonl(path):
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                ex = json.loads(line)
                yield ex

def write_jsonl(path, data):
    with open(path, "w") as f:
        for i, ex in enumerate(data):
            try:
                f.write(json.dumps(ex) + "\n")
            except TypeError as e:
                print(f"Error writing element at index {i}: {ex}")
                print(f"TypeError: {e}")

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

In [ ]:
#define your api_key
client2 = OpenAI(api_key=api_key, timeout=60)
def handle_completion(client2, message, response_format, max_tokens=512, temperature=1.0, top_p=0.7, seed=0):
    chat_completion = client2.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=message,
    max_tokens=max_tokens,
    temperature=temperature,
    top_p=top_p,
    seed=seed,
    response_format=response_format
    )

    extracted_data = chat_completion.choices[0].message.content
    return extracted_data

def handle_completion2(client2, message, max_tokens=512, temperature=1.0, top_p=0.7, seed=0):
    chat_completion = client2.chat.completions.create(
    model="gpt-4o-2024-08-06",
    messages=message,
    max_tokens=max_tokens,
    temperature=temperature,
    top_p=top_p,
    seed=seed,
    )

    extracted_data = chat_completion.choices[0].message.content
    return extracted_data

In [ ]:
problems_demo=[]
import pandas as pd
d=pd.read_excel('problems_demo_2.xlsx')
for i,row in d.iterrows():
    problems_demo.append({'problem':row['Problem'],'frame':row['Frame'],'rationale':row['Rationale'],'valid':row['valid'],'modified problem':row['modified problem'],'articulated':row['articulate'],'Comment':row['Comment']})
write_jsonl('problems_demo.jsonl',problems_demo)

Trying out json schema: Articulated and validity. Should we modify or throw away invalid/improperly articulated frames.

In [ ]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "valid_find",
        "strict": True,
        "schema": {
            "type": "object",
            "properties": {
                "validity": {
                    "type": "string",
                    "enum":["yes","no"]
                },
                "modified_problem": {
                    "type": "string",

                },
                "articulated": {
                    "type": "string",
                    "enum":["yes","no"]
                }
            },
            "required": ["validity","modified_problem","articulated"],
            "additionalProperties": False
        }
    }
}



In [ ]:
#consent
#alimony
comments={'both':'The problem is valid and properly articulated.','valid':'The problem is not valid.','art':'The problem is valid but not properly articulated.'}
#only check for problems other than obj, vio, ster, shaming, and other.
sys = '''You are an expert linguistic assistant.
Frames of communication select particular aspects of an issue and make them salient in communicating a message.
Frames of communication are ubiquitous in social media discourse and can impact how people understand issues and, more importantly, how they form their opinions.'''
changes=[]

message=[{"role": "system", "content": sys},]

demos=list(read_jsonl('problems_demo.jsonl'))
demos=demos[:10]
#take maximum 1 frames for each problem

'''for i,d in enumerate(demos):
  #if i==0:
  user_p=f"You will be given a frame and the problem associated with the frame. Assume you are a woman, is the problem correctly discovered and articulated with respect to the frame? If no, regenerate the problem."
  user_ps=f"The problem is {d['problem']}. 1.a) The Frame associated with the problem is: {d['frame']}. 1.b) The rationale generated for the frame is: {d['rationale']}"
  user_p=user_p+'\n'+user_ps
  message.append({"role":"user","content":[{"type":"text","text":user_p}]})
  if d['Comment']==comments['both']:
    message.append({"role":"assistant","content":[{"type":"json_schema","json_schema":{"validity": d['Comment']}}]})
  else:
    message.append({"role":"assistant","content":[{"type":"json_schema","json_schema":{"validity": d['Comment'], "modified problem": d["modified problem"]}}]})
'''
for i,d in enumerate(demos):
  #if i==0:
  user_p=f"You will be given a frame and the problem associated with the frame. From a woman's perspective, is the problem correctly discovered?. Is the problem articulated concisely with respect to the frame? Modify the problem only if the answer to any of these two questions is no. The modified problem should be specific and should not contain more than three words."
  user_ps=f"The problem is {d['problem']}. 1.a) The Frame associated with the problem is: {d['frame']}. 1.b) The rationale generated for the frame is: {d['rationale']}"
  user_p=user_p+'\n'+user_ps
  message.append({"role":"user","content":[{"type":"text","text":user_p}]})
  if d['Comment'] == comments['both']:
        validity_content = {"validity": d['valid'], 'articulated':d['articulated'], "modified_problem": ""}
  else:
        validity_content = {"validity": d['valid'],'articulated':d['articulated'], "modified_problem": d["modified problem"]}
  message.append({"role": "assistant", "content": [{"type": "text", "text": str(validity_content)}]})
message1=message.copy()
for problem in problems:
  if problem in ['objectification','violence','stereotyping','shaming','patriarchy','Other']:
    continue
  user_p=f"You will be given a frame and the problem associated with the frame. From a woman's perspective, is the problem correctly discovered?. Is the problem articulated concisely with respect to the frame? Modify the problem only if the answer to any of these two questions is no. The modified problem should be specific and should not contain more than three words."
  user_ps=f"The problem is {problem}. 1.a) The Frame associated with the problem is: {prblm_dict[problem][0]} 1.b) The rationale generated for the frame is: {reason_dict[problem][0]}"
  user_p=user_p+'\n'+user_ps
  message.append({"role":"user","content":[{"type":"text","text":user_p}]})
  #print(message)
  response=handle_completion(client2, message,response_format)
  changes.append({'problem':problem, 'response':response})
  message=message1.copy()
  print(response)



In [ ]:
write_jsonl('valid_art_1.jsonl',changes)

In [ ]:

#assuming problem is present only in the beginning (the problem of - arises)
import re
changes=list(read_jsonl('valid_art_1.jsonl'))

prblm_dict={}
reason_dict={}
with open('problems.txt', 'r') as f:
    problems = f.read().splitlines()
    problems=[problem.split(':')[0] for problem in problems]
for problem in problems:
    problem2='problem of '+ problem + ' arises'
    frame_list,reason_list=get_frame_from_problem(problem2)
    prblm_dict[problem]=frame_list
    reason_dict[problem]=reason_list


for change in changes:
  change['response']=json.loads(change['response'])
  modified_problem=change['response']['modified_problem']

  if modified_problem=='':
    continue
  
  temp=reason_dict[change['problem']]
  ftemp=prblm_dict[change['problem']]
  del prblm_dict[change['problem']]
  del reason_dict[change['problem']]

  mod_reasons=[]
  for reason in temp:
    #print(reason)

    new_text = re.sub(change['problem'], modified_problem, reason, flags=re.IGNORECASE)
    mod_reasons.append(new_text)
    #print(new_text)
  if modified_problem in prblm_dict.keys():
    prblm_dict[modified_problem].extend(ftemp)
    reason_dict[modified_problem].extend(mod_reasons)
  else:
     prblm_dict[modified_problem]=ftemp
     reason_dict[modified_problem]=mod_reasons


  problems = [modified_problem if problem == change['problem'] else problem for problem in problems]


In [ ]:

# Merging the dictionaries
merged_dict = {}
for problem in prblm_dict:


    frames = prblm_dict[problem]
    reasoning = reason_dict.get(problem, [])

    # Pair frames and reasoning
    merged_list = [{"frame": frame, "reasoning": reason} for frame, reason in zip(frames, reasoning)]


    # Add to merged_dict
    merged_dict[problem] = merged_list



Paraphrases

After finding valid and correctly articulated problems, modify them and find parapharses.

In [ ]:
#paraphrase module

#A sentence can have multiple paraphrases. Choose the one with more frames.

paraphrases=[]
sys = '''You are an expert linguistic assistant.
Frames of communication select particular aspects of an issue and make them salient in communicating a message.
Frames of communication are ubiquitous in social media discourse and can impact how people understand issues and, more importantly, how they form their opinions.'''


demos={'commodification': "Paraphrases(commodification, reducing women's worth to their bodies, Objectification)",'appearance-based discrimination':"Paraphrases(appearance-based discrimination, appearance-based judgment, appearance-based derogation, appearance-based value)", 'gender bias':"Paraphrases(gender bias, gender inequality, gender superiority, gender hierarchy, resistance to gender equality)"}
message=[{"role": "system", "content": sys},]

for key, value in demos.items():
  user_p='''Here is a exhaustive list of problems extracted from the reasoning generated alongside frames: '''
  for i,p in enumerate(problems):
      user_p=user_p+str(i+1)+') '+p+' '
  user_p2='''For an input problem x, compare it to the other problems from the list and generate all the paraphrases in the format if they exist: Paraphrases(problem x, problem y, problem z, so on).'''
  user_p=user_p+'\n'+user_p2
  user_ps=f"The problem is {key}."
  user_p=user_p+'\n'+user_ps
  message.append({"role":"user","content":[{"type":"text","text":user_p}]})
  message.append({"role":"assistant","content":[{"type":"text","text":value}]})



message1=message.copy()
for problem in problems:
  if problem in ['objectification','violence','stereotype','shaming','patriarchy','Other']:
    continue
  user_p='''Here is a exhaustive list of problems extracted from the reasoning generated alongside frames: '''
  for i,p in enumerate(problems):
    user_p=user_p+str(i+1)+') '+p+' '
  user_p2='''For an input problem x, compare it to the other problems from the list and generate all the paraphrases in the format if they exist: Paraphrases(problem x, problem y, problem z, so on).'''
  user_p=user_p+'\n'+user_p2
  user_ps=f"The problem is {problem}."
  user_p=user_p+'\n'+ user_ps
  message.append({"role":"user","content":[{"type":"text","text":user_p}]})
  response=handle_completion2(client2, message)
  #response = completion.choices[0].message.content
  paraphrases.append({'problem':problem, 'response':response})
  message=message1.copy()
  print(response)




In [ ]:
paraphrases=list(read_jsonl('pparaphrase_1.jsonl'))
print(len(paraphrases))
paraphrases_list=[]
for paraphrase in paraphrases:

  if 'Paraphrases(' not in paraphrase['response']:
    continue
  pt=(paraphrase['response'].split('Paraphrases(')[1]).split(')')[0].split(', ')
  if len(pt)>1:
    paraphrases_list.append(tuple(pt))
print(len(paraphrases_list))

500
479


In [ ]:
write_jsonl('pparaphrase_1.jsonl',paraphrases)


In [ ]:

# Function to merge paraphrased problems
def merge_paraphrased_problems(merged_dict, paraphrases):

    for group in paraphrases:

        existing_problems = [p for p in group if p in merged_dict]
        if  len(existing_problems)==0:
            continue
        #what if len==1.

        main_problem = max(existing_problems, key=lambda p: len(merged_dict[p]))

        # Collect all frames and reasoning from the other existing problems
        for problem in existing_problems:
            if problem != main_problem:
                merged_dict[main_problem].extend(merged_dict[problem])
                del merged_dict[problem]  # Remove the merged problem

        #Optional: remove duplicates from the merged list
        seen = set()
        merged_dict[main_problem] = [
            d for d in merged_dict[main_problem]
            if (d['frame'], d['reasoning']) not in seen and not seen.add((d['frame'], d['reasoning']))
        ]

    return merged_dict

# Apply the function
updated_merged_dict = merge_paraphrased_problems(merged_dict, paraphrases_list)


In [ ]:
frame_file = 'predictions_whole/articulations-unique.jsonl'
total_files = list(read_jsonl(frame_file))
frames = [j['text'] for j in total_files]

# Iterate through frames and problems in the dictionary
for i, frame in enumerate(frames):
    for key, value in updated_merged_dict.items():
        for j, v in enumerate(value):
            if v['frame'] == frame:
                updated_merged_dict[key][j]['frame_id'] = i + 1
missing_frame_id_count = 0

for key, value in updated_merged_dict.items():
    if any('frame_id' not in v for v in value):
        missing_frame_id_count += 1

print(f"Number of keys with missing frame_id: {missing_frame_id_count}")

Number of keys with missing frame_id: 0


In [ ]:
with open('problems_final.txt', 'w') as f:
    for p in updated_merged_dict.keys():
      f.write(p+'\n')


Sub problems

In [ ]:
#change demos based on updated problems
#skip the root problems

subproblems=[]
sys = '''You are an expert linguistic assistant.
Frames of communication select particular aspects of an issue and make them salient in communicating a message.
Frames of communication are ubiquitous in social media discourse and can impact how people understand issues and, more importantly, how they form their opinions.'''

demos={'sexual harassment': "subproblem(Sexual harassment, Violence)", 'dehumanization': "subproblem(dehumanization, Objectification)", "ageism": "subproblem(ageism, Stereotype)", "trivializing women's issues":"subproblem(trivializing, trivializing women's issues)",}
message=[{"role": "system", "content": sys},]

for key, value in demos.items():
  user_p='''Here is a exhaustive list of problems extracted from the reasoning generated alongside frames: '''
  for i,p in enumerate(problems):
    user_p=user_p+str(i+1)+') '+p+' '
  user_p2='''For an input problem x, compare it to the other problems from the list and find out it's closest parent problem (if it exists). The output should be in the format: subproblem(problem x, problem y).'''
  user_p=user_p+'\n'+user_p2
  user_ps=f"The problem is {key}."
  user_p=user_p+'\n'+user_ps
  message.append({"role":"user","content":[{"type":"text","text":user_p}]})
  message.append({"role":"assistant","content":[{"type":"text","text":value}]})

message1=message.copy()
cnt=0
for problem in problems:
  if problem in ['objectification','violence','stereotyping','shaming','patriarchy','Other']:
    continue
  user_p='''Here is a exhaustive list of problems extracted from the reasoning generated alongside frames: '''
  for i,p in enumerate(problems):
    user_p=user_p+str(i+1)+') '+p+' '
  user_p2='''For an input problem x, compare it to the other problems from the list and find out it's closest parent problem (if it exists). The output should be in the format: subproblem(problem x, problem y).'''
  user_p=user_p+'\n'+user_p2
  user_ps=f"The problem is {problem}."
  user_p=user_p+'\n'+ user_ps
  message.append({"role":"user","content":[{"type":"text","text":user_p}]})
  response=handle_completion2(client2, message)
  subproblems.append({'problem':problem, 'response':response})
  message=message1.copy()
  print(response)




In [ ]:
write_jsonl('subproblems_1.jsonl',subproblems)

In [ ]:
import json
from collections import defaultdict

# Function to read frames multiple and convert it into key plus list of frmes.
def read_from_jsonl_ind(filename):
    data = defaultdict(list)

    with open(filename, 'r') as file:
        for line in file:
            record = json.loads(line.strip())
            problem = record.pop("problem")
            data[problem].append(record)

    # Convert defaultdict back to a regular dict
    return dict(data)


In [ ]:
subproblems=list(read_jsonl('subproblems_1c.jsonl'))

subproblems_list=[]
for subproblem in subproblems:

  if 'subproblem(' not in subproblem['response']:
    continue
  pt=(subproblem['response'].split('subproblem(')[1]).split(')')[0].split(', ')
  if len(pt)>1:
    subproblems_list.append(tuple(pt))
print(len(subproblems_list))

103


In [ ]:
from collections import defaultdict
problems=list(updated_merged_dict.keys())
hierarchical_graph = defaultdict(list)

for child, parent in subproblems_list:
    hierarchical_graph[parent].append(child)

# Dictionaries to store the relationships
parent_to_children = defaultdict(list)
child_to_parent = {}

# Build the graph relationships
for child, parent in subproblems_list:
    parent_to_children[parent].append(child)
    child_to_parent[child] = parent

# Optional: Add isolated problems without parents (nodes with no incoming edges)
for problem in problems:
    if problem not in hierarchical_graph and problem not in [child for child, parent in subproblems_list]:
        hierarchical_graph[problem] = []

# Function to recursively print the hierarchy
def print_hierarchy(graph, parent, level=0):
    print("  " * level + parent)
    for child in graph[parent]:
        print_hierarchy(graph, child, level + 1)

# Find root nodes (problems that have no parents)
roots = [problem for problem in hierarchical_graph if problem not in [child for child, parent in subproblems_list]]

# Print the hierarchical graph
for root in roots:
    print_hierarchy(hierarchical_graph, root)


patriarchy
  enforced gender norms
    gender essentialism
    constrained gender identity
  double standards
    conditional respect
  undermining women's capabilities
    demeaning aspirations
  male validation
  gender superiority
  toxic masculinity
  dismissing women's rights
    undermining advocacy
      undermining women's rights movements
  historical misogyny
  subjugation
  patriarchal control
    mistrust
      deception
      conspiracy thinking
      betrayal
        infidelity
      commitment phobia
    possessiveness
    gatekeeping
    coercion
    ownership
    policing women's bodies
    forced marriage
objectification
  sexualization
    unrealistic beauty standards
    sexual innuendo usage
    sexual entitlement
    age-related sexualization
  dehumanization
    intellectual degradation
    disposability
    demonization
      fearmongering
  ranking women
discrimination
  ableism
  transphobia
  exclusion
  workplace discrimination
    professional misconduct
  